OCR

[algospot의 OCR 링크](https://algospot.com/judge/problem/read/OCR)
  
### 문제  
광학 문자 인식(Optical Character Recognition)은 사람이 쓰거나 기계로 인쇄한 글자를 스캔한 이미지를 다시 기계가 읽을 수 있는 문자로 변환하는 과정을 말합니다. OCR 알고리즘들은 대개 수많은 필기 샘플을 통계적으로 분석하고 패턴을 찾아내어 각 단어들을 인식하곤 합니다. 하지만 단순히 각 단어들을 개별적으로 인식하기보다, 단어의 분포나 문법 등을 고려하면 더 나은 결과를 얻을 수 있는 경우가 많습니다. 이 문제에서는 과거 자료로부터 추출한 정보를 이용해 문자 인식의 정확도를 높여 봅시다.

과거에 인식했던 수많은 문장들을 분석해 원본 문장의 형태를 파악하려고 합니다. 이 작업을 위해 우선 과거 자료에 출현하는 모든 단어의 목록을 만든 뒤, 각 단어가 문장의 첫 단어로 사용된 비율을 계산했습니다. 그리고 각 단어 쌍에 대해, 한 단어가 다른 단어 다음에 출현할 확률을 계산했습니다. 이때 우리가 인식해야 할 원본 문장은 과거 자료와 똑같은 분포를 가진다고 가정합시다. 달리 말해 이 확률 테이블만 있으면 어떤 원본 문장이 출현할 확률을 정확히 계산할 수 있다고 가정한다는 얘깁니다.

우리의 문자 인식 알고리즘은 원문 그림을 여러 조각으로 쪼갠 후 각 조각을 비슷해 보이는 단어로 분류합니다. 이 분류하는 알고리즘을 분류기(classifier)라고 부릅니다. 이 분류기는 완벽하지 않기 때문에 특정 단어를 다른 단어로 잘 인식할 수도 있습니다. 예를 들어 boy라는 단어를 buy나 bay로 인식할 수 있다는 이야기입니다. 수많은 예제 입력에 대해 분류기를 시험하여, 각 단어가 적힌 조각을 분류기에 입력했을 때 어떤 출력을 얻을 수 있는지, 그리고 각각의 확률은 얼마였는지를 계산했습니다. 예를 들어 분류기에 실제 boy라고 씌어 있는 조각을 입력했을 때, 정확하게 boy로 인식할 확률은 0.7, bay일 확률은 0.25, buy일 확률은 0.04, bye일 확률은 0.01이었다는 식입니다.

이와 같은 정보들을 이용하면 좀더 나은 문자 인식을 할 수 있습니다. 각 조각을 앞에서 예로 든 분류기를 이용해 인식한 결과 "I am a bay."라는 문장을 결과로 얻었다고 합시다. 그런데 자료를 살펴보니 a 후에 bay가 올 확률은 얼마 없는 반면, a 후에 boy가 올 확률은 매우 컸다고 합시다. 우리의 분류기가 bay라고 인식한 조각이 사실은 boy일 확률이 0.25나 되기 때문에, 이 문장의 인식 결과를 "I am a boy."로 고치는 편이 더 올바른 분류일 것입니다.

어떤 문장을 단어별로 인식한 결과가 주어졌을 때, 원본일 조건부 확률이 가장 높은 문장을 찾아내는 프로그램을 작성하세요.  
  
### 출력  
한 문장마다 한 줄에 주어진 인식 결과에 대해 조건부 출현 확률이 가장 높은 문장을 출력합니다. 주어지는 입력에서 가장 확률이 높은 문장이 여러 개인 경우 어느 것을 출력해도 좋습니다.  

solution의    
words : 원문에 출현할 수 있는 단어 (1≤len(words)≤500)  
prob_first : 각 단어가 문장의 처음에 출현할 확률을 가지는 리스트  
T : i행 j열의 숫자 T[i, j]는 i번 단어의 다음 단어가 j번 단어일 확률을 가지는 행렬  
M : i행 j열의 숫자 M[i, j]는 i번 단어가 적힌 조각을 j번 단어로 분류할 확률을 가지는 행렬  
Q : 분류기로 인식한 문장들 [단어의 수, 분류기의 인식결과]을 가지는 리스트  
  
recognize_sentence의  
추가된  
word_num : 현재 인식한 문장의 개수  
prev_word : 바로 전에 오는 단어의 인덱스

In [1]:
def recognize_sentence(words, classified, prob_first, T, M, Q, word_num = 0, prev_word = 0):
    max_sentence = [] # 최대 확률을 가지는 문장 (단어의 인덱스의 리스트)
    max_prob = 0 # 최대 확률
    if word_num == Q[0]: # 만약 인식된 글자 수와 같다면 리턴 : basecase
        return [], 1
    
    for i in range(len(words)): # 원문에 있는 모든 단어에 대해서
        sentence = [] # 문장 저장할 리스트 (단어의 인덱스의 리스트)
        prob = 1 # 현재의 문장이 나올 확률
        if word_num == 0: # 만약 문장의 처음 오는 단어의 경우
            sentence.append(i) # 단어 일단 추가 후
            prob *= prob_first[i] # 각 단어가 문장의 처음에 출현할 확률 곱해줌
        else: # 문장의 처음에 오는 단어가 아니라면
            prob *= T[prev_word][i] # 이전 단어에서 현재 추가할 단어가 올 확률 곱
            sentence.append(i) # 단어 추가
        prob *= M[i][classified[word_num]] # 추가된 단어에 대해서 현재 분류된 단어로 분류될 확률 곱
        if prob == 0: continue # 만약 현재 확률이 0일 경우 더 계산과정을 거치지 않는다.
        temp_sentence, temp_prob = recognize_sentence(words, classified, prob_first, T, M, Q, word_num+1, i)
        # 재귀로서 문장의 끝까지 현재의 과정을 진행
        prob *= temp_prob # 뒤에 오는 단어에 대한 확률을 곱해줌.
        sentence += temp_sentence # 뒤에 오는 단어들을 리스트에 더해줌
        if max_prob < prob: # 만약 현재의 최대 확률보다 크다면
            max_prob = prob # 확률 저장
            max_sentence = sentence # 문장 저장
    return max_sentence, max_prob # 문장과 확률 반환

def solution(words, prob_first, T, M, Q):
    answer = "" # 답을 저장할 변수
    classified = [] # 분류된 문장의 단어 리스트
    for i in Q[1].split():
        classified.append(words.index(i)) # 문류된 문장에 대해서 인덱스로 저장해줌 
    answer_sentence, _ = recognize_sentence(words, classified ,prob_first, T, M, Q)
    # 최대 확률을 지니는 문장에 대해서 반환
    for i in answer_sentence:
        answer += words[i] + " "
    return answer # 답 반환

In [2]:
words = ["I", "am", "a", "boy", "buy"]
prob_first = [1.0, 0.0, 0.0, 0.0, 0.0]
T = [[0.1, 0.6, 0.1, 0.1, 0.1],
[0.1, 0.1, 0.6, 0.1, 0.1],
[0.1, 0.1, 0.1, 0.6, 0.1],
[0.2, 0.2, 0.2, 0.2, 0.2],
[0.2, 0.2, 0.2, 0.2, 0.2]]
M = [[0.8, 0.1, 0.0, 0.1, 0.0],
[0.1, 0.7, 0.0, 0.2, 0.0],
[0.0, 0.1, 0.8, 0.0, 0.1],
[0.0, 0.0, 0.0, 0.5, 0.5],
[0.0, 0.0, 0.0, 0.5, 0.5]]
Q = [4, "I am a buy"]

In [3]:
solution(words, prob_first, T, M, Q)

'I am a boy '

In [4]:
words = ["I", "am", "a", "boy", "buy"]
prob_first = [1.0, 0.0, 0.0, 0.0, 0.0]
T = [[0.1, 0.6, 0.1, 0.1, 0.1],
[0.1, 0.1, 0.6, 0.1, 0.1],
[0.1, 0.1, 0.1, 0.6, 0.1],
[0.2, 0.2, 0.2, 0.2, 0.2],
[0.2, 0.2, 0.2, 0.2, 0.2]]
M = [[0.8, 0.1, 0.0, 0.1, 0.0],
[0.1, 0.7, 0.0, 0.2, 0.0],
[0.0, 0.1, 0.8, 0.0, 0.1],
[0.0, 0.0, 0.0, 0.5, 0.5],
[0.0, 0.0, 0.0, 0.5, 0.5]]
Q = [4, "I I a boy"]

In [5]:
solution(words, prob_first, T, M, Q)

'I am a boy '

In [6]:
words = ["I", "am", "a", "boy", "buy"]
prob_first = [1.0, 0.0, 0.0, 0.0, 0.0]
T = [[0.1, 0.6, 0.1, 0.1, 0.1],
[0.1, 0.1, 0.6, 0.1, 0.1],
[0.1, 0.1, 0.1, 0.6, 0.1],
[0.2, 0.2, 0.2, 0.2, 0.2],
[0.2, 0.2, 0.2, 0.2, 0.2]]
M = [[0.8, 0.1, 0.0, 0.1, 0.0],
[0.1, 0.7, 0.0, 0.2, 0.0],
[0.0, 0.1, 0.8, 0.0, 0.1],
[0.0, 0.0, 0.0, 0.5, 0.5],
[0.0, 0.0, 0.0, 0.5, 0.5]]
Q = [4, "I am am boy"]

In [7]:
solution(words, prob_first, T, M, Q)

'I am a boy '